# Fine-Tune Whisper For Even Speech Pakendorf Dataset with 🤗 Transformers on Even Pakendorf dataset

In this Notebook, we present a step-by-step guide on how to fine-tune Whisper
for Even ASR Pakendorf dataset using Hugging Face 🤗 Transformers. This is heavily based on a
more general [blog post](https://huggingface.co/blog/fine-tune-whisper) for fine-tuning Whisper.
For a more in-depth explanation of Whisper and tuning the reader is advised to refer to the blog post.

## Prepare Environment

Check python version - lower than 3.10 will make errors happen!

In [1]:
import sys

print(sys.version)

3.11.7 | packaged by Anaconda, Inc. | (main, Dec 15 2023, 18:05:47) [MSC v.1916 64 bit (AMD64)]


Then check GPU availability!

To get a GPU on Colab, click _Runtime_ -> _Change runtime type_, then change _Hardware accelerator_ from _CPU_ to one of the available GPUs, e.g. _T4_ (or better if you have one available). Next, click `Connect T4` in the top right-hand corner of your screen.

When running on local mashine check GPU specifications and CUDA availability via cells below. You can check whether your device is CUDA-compatible [here](https://developer.nvidia.com/cuda-gpus).

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Fri Jun 14 16:34:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.17                 Driver Version: 546.17       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  | 00000000:0A:00.0  On |                  N/A |
| 42%   33C    P8              11W / 170W |    510MiB / 12288MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

You can start locally with torch and CUDA using this [explanation](https://pytorch.org/get-started/locally/).

In [1]:
import torch

torch.cuda.is_available()

True

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets[audio]` to download and prepare our training data, alongside
`transformers` and `accelerate` to load and train our Whisper model.
We'll also require the `soundfile` package to pre-process audio files,
`evaluate` and `jiwer` to assess the performance of our model, and
`tensorboard` to log our metrics. Finally, we'll use `gradio` to build a
flashy demo of our fine-tuned model.

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio huggingface_hub

Linking the notebook to the Hub is straightforward - it simply requires entering your
Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [2]:
from huggingface_hub import notebook_login

notebook_login()

If widgets are not working correctly on your local machine you can save access token via cells below.

In [3]:
access_token='YOUR_ACCESS_TOKEN'

In [4]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token(access_token)

## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple.

In this Notebook dataset [tbkazakova/even_speech_pakendorf](https://huggingface.co/datasets/tbkazakova/even_speech_pakendorf) was used. It has ~10 hours of data, so it was split into train and test datasets in 1:3 proportion.

In [32]:
from datasets import load_dataset, DatasetDict

even_dsdict = load_dataset(
    "tbkazakova/even_speech_pakendorf", split="train",
    use_auth_token=True, trust_remote_code=True
).train_test_split(test_size=0.25)
even_dsdict

C:\Users\Александра\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Resolving data files:   0%|          | 0/7916 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 5936
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 1979
    })
})

## Prepare Feature Extractor, Tokenizer, Processor and Data

The ASR pipeline can be de-composed into three stages:

1. A feature extractor which pre-processes the raw audio-inputs
2. The model which performs the sequence-to-sequence mapping
3. A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

### Load WhisperFeatureExtractor

We'll load the feature extractor from the pre-trained checkpoint with the default values. If running locally be sure to choose `device_map` to run your computations on GPU. Check [HF docs](https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained) for more info.

In [33]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small", device_map='auto')

### Load WhisperTokenizer

For Even language, we can not load the pre-trained tokenizer of Whisper model since it does not exist. In this Notebook `russian` language was chosen as the one the model will run on.

Now we simply have to specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences.

In [34]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="ru", task="transcribe", device_map='auto')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [35]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="ru", task="transcribe", device_map='auto')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Prepare Data

Input audio from dataset needs to be downsampled to
16kHz for it to be passed to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [36]:
from datasets import Audio

even_dsdict = even_dsdict.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Even Pakendorf dataset will resample
it to the desired sampling rate:

In [39]:
even_dsdict["train"][0]

{'audio': {'path': 'C:\\Users\\Александра\\.cache\\huggingface\\datasets\\downloads\\a7609bb9ebdc276024d9d76731e60eae61a3f188d32a4bdfb09af19bf8bd6afe',
  'array': array([ 0.00079543,  0.0029001 ,  0.00428934, ..., -0.00049313,
         -0.0012945 ,  0.00044376]),
  'sampling_rate': 16000},
 'transcription': 'таду бакаркитникан бакаркитникан хан бэйил носэгчер яв кахандыл икер би чэ элэкэн эдэку икер гоникэн оддыв '}

Now we can write a method to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We remove all punctuation from transcriptions and encode them to label ids through the use of the tokenizer.

TokenizerWrapper was created to solve errors with feature extractor and tokenizer definitions - so that `tokenize_functions` only uses in-class-defined functions.

In [40]:
import re

class TokenizerWrapper:
    def __init__(self, tokenizer, feature_extractor):
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.punctuation_chars = '[\,\?\.\!\-\;\:\"\“\%\‘\”\'\«\»\–\‐\„]'
        self.replacer = re.sub
    
    def tokenize_function(self, batch):
        audio = batch["audio"]
        batch["input_features"] = self.feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
        
        batch["transcription"] = self.replacer(self.punctuation_chars, "", batch["transcription"]).lower()
        batch["labels"] = self.tokenizer(batch["transcription"]).input_ids
        return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [41]:
wrapper = TokenizerWrapper(tokenizer, feature_extractor)

In [42]:
even_dsdict = even_dsdict.map(wrapper.tokenize_function, remove_columns=even_dsdict.column_names["train"], num_proc=4)

Map (num_proc=4):   0%|          | 0/5936 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1979 [00:00<?, ? examples/s]

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it
to transcribe speech in Even (though using Russian as model language).

### Load a Pre-Trained Checkpoint

We'll start our fine-tuning run from the pre-trained Whisper `small` checkpoint,
the weights for which we need to load from the Hugging Face Hub. 

Don't forget to set `device_map` for local running on GPU.

In [43]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", device_map="auto")

We can disable the automatic language detection task performed during inference, and force the model to generate in Russian. To do so, we set the [langauge](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.language)
and [task](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.task)
arguments to the generation config. We'll also set any [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)
to None, since this was the legacy way of setting the language and
task arguments:

In [44]:
model.generation_config.language = "russian"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [45]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [46]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [55]:
import evaluate

metric_wer = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [56]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric_wer.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [49]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-pakendorf",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size in case of memory issues
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

C:\Users\Александра\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:

In [50]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=even_dsdict["train"],
    eval_dataset=even_dsdict["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

max_steps is given, it will override any value given in num_train_epochs


We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [51]:
processor.save_pretrained(training_args.output_dir)

[]

### Training

Training might take a lot of time (more than 5 hours) depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.

To launch training, simply execute:

In [52]:
trainer.train()

C:\Users\Александра\AppData\Roaming\Python\Python311\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
1000,0.661900,0.996137,86.273731
2000,0.098900,1.110092,68.814044
3000,0.011400,1.243196,68.464533
4000,0.004900,1.308649,67.987926


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}
C:\Users\Александра\AppData\Roaming\Python\Python311\site-packages\torch\utils\checkpoint.py:464: UserWarn

TrainOutput(global_step=4000, training_loss=0.4152448774985969, metrics={'train_runtime': 21299.2516, 'train_samples_per_second': 3.005, 'train_steps_per_second': 0.188, 'total_flos': 1.846946562048e+19, 'train_loss': 0.4152448774985969, 'epoch': 10.78167115902965})

The training results can now be uploaded to the Hub.

In [53]:
kwargs = {
    "dataset_tags": "tbkazakova/even_speech_pakendorf",
    "dataset": "Even Speech Pakendorf",
    "dataset_args": "config: ru, split: train",
    "language": "ru",
    "model_name": "Whisper Small for Even Pakendorf",
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [54]:
trainer.push_to_hub(**kwargs)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}


CommitInfo(commit_url='https://huggingface.co/tanya-kta/whisper-small-pakendorf/commit/d51681605a0199e4d0609f3937c7202650b67766', commit_message='End of training', commit_description='', oid='d51681605a0199e4d0609f3937c7202650b67766', pr_url=None, pr_revision=None, pr_num=None)

### Computing CER

We load CER from `evaluate`. Then predict values using Trainer and pass them through code from `compute_metrics`.

In [57]:
metric_cer = evaluate.load("cer")

In [ ]:
predictions, label_ids, metrics = trainer.predict(even_dsdict["test"])

In [60]:
label_ids[label_ids == -100] = tokenizer.pad_token_id

pred_str = tokenizer.batch_decode(predictions, skip_special_tokens=True)
label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

cer = 100 * metric_cer.compute(predictions=pred_str, references=label_str)
metrics["test_cer"] = cer
metrics

{'test_loss': 1.3086488246917725,
 'test_wer': 67.98792596711415,
 'test_runtime': 873.535,
 'test_samples_per_second': 2.266,
 'test_steps_per_second': 0.284,
 'test_cer': 28.463889375986373}